In [1]:
from langchain_community.chat_models import ChatZhipuAI
from dotenv import load_dotenv,find_dotenv
import os 

In [17]:
_ = load_dotenv(find_dotenv())
chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5
)

In [18]:
template_string = """用{style}格式,翻译一下这个用三个反引号分隔的文本 .
```{orginal_txt}```
"""

In [19]:
from langchain.prompts import ChatPromptTemplate

In [20]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [21]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['orginal_txt', 'style'], template='用{style}格式,翻译一下这个用三个反引号分隔的文本 .\n```{orginal_txt}```\n')

In [22]:
prompt_template.messages[0].prompt.input_variables

['orginal_txt', 'style']

In [23]:
style = """
中国大陆普通话 \
日常口语 \
平常和礼貌的语气
"""

In [32]:
orginal_txt = """
 ぺ畱ぅ很9的頭發能ゐ尒説剪就剪，愛ぅ很9的尒也能説忘就忘。
"""

In [33]:
customer_messages = prompt_template.format_messages(style=style,orginal_txt=orginal_txt)

In [34]:
type(customer_messages)

list

In [35]:
type(customer_messages[0])

langchain_core.messages.human.HumanMessage

In [36]:
customer_messages[0]


HumanMessage(content='用\n中国大陆普通话 日常口语 平常和礼貌的语气\n格式,翻译一下这个用三个反引号分隔的文本 .\n```\n ぺ畱ぅ很9的頭發能ゐ尒説剪就剪，愛ぅ很9的尒也能説忘就忘。\n```\n')

In [37]:
customer_resp = chat.invoke(customer_messages)

In [38]:
customer_resp

AIMessage(content='```\n留着很长的头发你说剪就剪，深爱着你的我也能说忘就忘。\n``` \n\n（这里做了一点语序上的调整，以使其更符合汉语的表达习惯。）', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 75, 'total_tokens': 117}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-58bbc7eb-64d8-4b52-88cc-82205fdb36c2-0')

In [39]:
customer_resp.content

'```\n留着很长的头发你说剪就剪，深爱着你的我也能说忘就忘。\n``` \n\n（这里做了一点语序上的调整，以使其更符合汉语的表达习惯。）'

style = """
中国香港粤语 \
日常口语俚语 
"""
orginal_txt = """
 没什么大不了的事，看开一点吧
"""
customer_messages = prompt_template.format_messages(style=style,orginal_txt=orginal_txt)
customer_resp = chat.invoke(customer_messages)
customer_resp.content

In [48]:
constomer_review = """\
犯罪不分大人小孩，都是坏人。\
单身父亲李长峰（王千源 饰）在女儿被几名未成年人凌虐死亡后，绝望地踏上了复仇之路，他似乎想用自己的方式让少年恶魔们付出应有的代价。面对李长峰和警方的追踪，这群少年犯会得到何种惩罚？\
影片根据东野圭吾同名小说改编。\
"""

review_template = """\
请根据下面的文本，提取信息：

演员:出演这部电影的演员
剧情:电影的剧情，故事情节
原著:是改编自哪一部电影或者图书

将输出的文本转成JSON格式,按下面的键值：
演员
剧情
原著

text:{text}
"""

In [49]:
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='请根据下面的文本，提取信息：\n\n演员:出演这部电影的演员\n剧情:电影的剧情，故事情节\n原著:是改编自哪一部电影或者图书\n\n将输出的文本转成JSON格式,按下面的键值：\n演员\n剧情\n原著\n\ntext:{text}\n'))])

In [52]:
messages = prompt_template.format_messages(text=constomer_review)
customer_resp = chat.invoke(messages)
print(customer_resp.content)

以下是提取的信息，按照您提供的键值以JSON格式组织：

```json
{
  "演员": "王千源",
  "剧情": "犯罪不分大人小孩，都是坏人。单身父亲李长峰在女儿被几名未成年人凌虐死亡后，绝望地踏上了复仇之路，他似乎想用自己的方式让少年恶魔们付出应有的代价。面对李长峰和警方的追踪，这群少年犯会得到何种惩罚？",
  "原著": "东野圭吾同名小说"
}
```

请注意，由于原始文本中没有提供原著具体的书名，所以在此JSON中“原著”字段只是简单地引用了“东野圭吾同名小说”。如果需要更详细的信息，可能需要进一步核实原著的确切名称。


In [53]:
type(customer_resp.content)

str

In [58]:
from langchain.output_parsers import ResponseSchema,StructuredOutputParser

In [59]:
actor_schema = ResponseSchema(name="演员",description="出演这部电影的演员")
story_schema = ResponseSchema(name="剧情",description="电影的剧情，故事情节")
ref_schema = ResponseSchema(name="原著",description="是改编自哪一部电影或者图书")

response_schemas = [actor_schema,story_schema,ref_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [62]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"演员": string  // 出演这部电影的演员
	"剧情": string  // 电影的剧情，故事情节
	"原著": string  // 是改编自哪一部电影或者图书
}
```


In [63]:
review_template_2 = """\
请根据下面的文本，提取信息：

演员:出演这部电影的演员
剧情:电影的剧情，故事情节
原著:是改编自哪一部电影或者图书

text:{text}

{format_instructions}
"""

In [64]:
prompt_template2 = ChatPromptTemplate.from_template(review_template_2)
prompt_template2

ChatPromptTemplate(input_variables=['format_instructions', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='请根据下面的文本，提取信息：\n\n演员:出演这部电影的演员\n剧情:电影的剧情，故事情节\n原著:是改编自哪一部电影或者图书\n\ntext:{text}\n\n{format_instructions}\n'))])

In [65]:
messages = prompt_template.format_messages(text=constomer_review,format_instructions=format_instructions)
customer_resp = chat.invoke(messages)
print(customer_resp.content)

以下是提取的信息，以JSON格式表示：

```json
{
  "演员": "王千源",
  "剧情": "犯罪不分大人小孩，都是坏人。单身父亲李长峰在女儿被几名未成年人凌虐死亡后，绝望地踏上了复仇之路，他似乎想用自己的方式让少年恶魔们付出应有的代价。面对李长峰和警方的追踪，这群少年犯会得到何种惩罚？",
  "原著": "东野圭吾同名小说"
}
```

请注意，原著字段中只提到了作者和小说的同名性质，但没有具体的书名。如果需要更详细的书名，需要进一步的信息来完善这一字段。


In [66]:
output_dict = output_parser.parse(customer_resp.content)
output_dict

{'演员': '王千源',
 '剧情': '犯罪不分大人小孩，都是坏人。单身父亲李长峰在女儿被几名未成年人凌虐死亡后，绝望地踏上了复仇之路，他似乎想用自己的方式让少年恶魔们付出应有的代价。面对李长峰和警方的追踪，这群少年犯会得到何种惩罚？',
 '原著': '东野圭吾同名小说'}

In [67]:
output_dict.get("演员")

'王千源'

In [68]:
output_dict.get("剧情")

'犯罪不分大人小孩，都是坏人。单身父亲李长峰在女儿被几名未成年人凌虐死亡后，绝望地踏上了复仇之路，他似乎想用自己的方式让少年恶魔们付出应有的代价。面对李长峰和警方的追踪，这群少年犯会得到何种惩罚？'